In [4]:
import folium
import requests
from google.transit import gtfs_realtime_pb2
import pandas as pd
import numpy as np
import datetime
from datetime import date

In [5]:
# Step-1 : Setting up Main API
API_KEY="8WcAOAQxN9pvZ8m4wZyJ3wLk4uMK4lb9"
API_URL="https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=8WcAOAQxN9pvZ8m4wZyJ3wLk4uMK4lb9"

In [6]:
# Step-2 : Fetching of Data
response=requests.get(API_URL)
if response.status_code != 200:
    raise Exception(f"Error ! {response.status_code}:{response.text}") #Mostly 405 Occurs

In [7]:
# Step-3 Data Parsing To Protobuf
feed=gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

169708

In [8]:
import datetime

data=[]
for entity in feed.entity:
    if entity.HasField('vehicle'):
        vehicle = entity.vehicle
        data.append({
            "VehicleID": vehicle.vehicle.id,
            "TripID": vehicle.trip.trip_id,
            "RouteID": vehicle.trip.route_id,
            "Latitude": vehicle.position.latitude,
            "Longitude": vehicle.position.longitude,
            "Timestamp": datetime.datetime.fromtimestamp(vehicle.timestamp) if vehicle.timestamp else None
        })


API is of 2 Types ::

In [9]:
df=pd.DataFrame(data)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1654 entries, 0 to 1653
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   VehicleID  1654 non-null   object        
 1   TripID     1654 non-null   object        
 2   RouteID    1654 non-null   object        
 3   Latitude   1654 non-null   float64       
 4   Longitude  1654 non-null   float64       
 5   Timestamp  1654 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 77.7+ KB


In [11]:
# Step-5 Map Visualization
n=folium.Map(location=[28.64857000,77.21895000],zoom_start=11)
for _, row in df.iterrows():
    folium.Marker(
        location=[row["Latitude"],row["Longitude"]],
        popup=f"Route : {row['RouteID']}<br>Vehicle : {row["VehicleID"]}<br>Time:{row['Timestamp']}",
        icon=folium.Icon(color='blue',icon='Metro')
    ).add_to(n)

In [12]:
n.save("Delhi Transport Mapping.html")
print("Map Saved Sucessfully !! in the Directory of Code")

Map Saved Sucessfully !! in the Directory of Code
